In [6]:
import os
import sys
import time
import napari
import pickle
import h5py
import numpy as np
import pandas as pd

from skimage import measure
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.morphology import white_tophat, disk

from napari.qt.threading import thread_worker

from itertools import product

from tifffile import imread,imsave,TiffFile
import matplotlib.pyplot as plt
import seaborn as sn

from nd2reader import ND2Reader
import btrack

In [2]:
myFov = '05'

myDirIm = r'Z:\Wayne\20210618_RPE_p21_cycD1_DHB_H2B\tiffs'
myFile = r'20210618_RPE_p21_cycD1_DHB_H2B_series_05_ch_01.tif'

myDirTracks = r'Z:\Wayne\20210618_RPE_p21_cycD1_DHB_H2B\tracking\05'
pointDir = r'Z:\Wayne\20210618_RPE_p21_cycD1_DHB_H2B\tracking\05_napari'

mitosisPointsPath = r'mitosis_points'
donePointsPath = r'done_points'
swapPointsPath = r'swap_points'

In [3]:
try:
    os.mkdir(pointDir)
    print('Directory for saving created.')
except(FileExistsError):
    print('Directory exists.')

Directory exists.


In [4]:
modelPath = r'D:\BARC\Sonja_2021\b_track\cell_config.json'

btrack_init_path = os.path.join(pointDir,f'{myFov}_tracks_init.h5')
btrack_mod_path = os.path.join(pointDir,f'{myFov}_tracks_mod.h5')

## Read in the tracking channel

In [5]:
%%time

# read in image

myIm = imread(os.path.join(myDirIm,myFile))
myIm.shape

TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'


Wall time: 53.6 s


(361, 2765, 2765)

## Read in signal channels

In [7]:
%%time

total_z = 2888 # because of how it is saved, it puts all frames of this channel from 8 fovs together 8x361=2888
fov_number = 8
myChannel = 1

imReader = ND2Reader(r'Z:\Wayne\20210618_RPE_p21_cycD1_DHB_H2B\20210618_RPE_p21_cycD1_DHB_H2B.nd2')

myFov = 5
 
myIm_signal = []

for i in np.arange(myFov,total_z,fov_number):

    temp = imReader.get_frame_2D(c=myChannel,z=i)
    myIm_signal.append(temp)

myIm_signal = np.array(myIm_signal).astype('uint16')

C:\ProgramData\Anaconda3\envs\cellpose\lib\site-packages\nd2reader\raw_metadata.py:171: UserWarning: Z-levels details missing in metadata. Using Z-coordinates instead.
  warnings.warn("Z-levels details missing in metadata. Using Z-coordinates instead.")
C:\ProgramData\Anaconda3\envs\cellpose\lib\site-packages\nd2reader\common_raw_metadata.py:99: RuntimeWarning: Reported average frame interval (600016.2 ms) doesn't match the set interval (600000.0 ms). Using the average now.
  warnings.warn(message % (avg_interval, interval), RuntimeWarning)


Wall time: 4min 18s


## Read in tracks in a form of labels

In [6]:
# read in labels as tracks

myLabels =[]

fovFiles = [x for x in os.listdir(myDirTracks) if f'series_{myFov}_' in x]

for myFile in fovFiles:
    
    # read a mask
    labels = imread(os.path.join(myDirTracks,myFile))
    
    myLabels.append(labels)
    
inProgressTracks = np.array(myLabels)

# create an empty contained for accepted tracks
acceptedTracks = inProgressTracks.copy()
acceptedTracks[:] = 0

In [7]:
# read in annotation points if present
try:
    donePoints = np.load(os.path.join(pointDir,donePointsPath+'.npy'))
    print('Accepted tracks loaded.')
except(FileNotFoundError):
    donePoints=np.array([])
    print('No accepted tracks found.')
    
    
try:
    mitosisPoints = np.load(os.path.join(pointDir,mitosisPointsPath+'.npy'))
    print('Mitosis points loaded.')
except(FileNotFoundError):
    mitosisPoints=np.array([])
    print('No mitosis points found.')
    
swapPoints = np.array([])

No accepted tracks found.
No mitosis points found.


## Divide the tracks into accepted and in progress

In [8]:
if len(donePoints) > 0:
    
    for myPoint in donePoints:
    
        # get the track that was marked as done
        myTrackNum = inProgressTracks[tuple(myPoint)]

        acceptedTracks[inProgressTracks==myTrackNum]=myTrackNum

        inProgressTracks[inProgressTracks==myTrackNum]=0

## Get latest tracking

In [198]:
# check if modified tracking exists

try:
    f = open(btrack_mod_path.replace('h5','pkl'),'rb')
    
    print('Opening corrected tracking.')

except(FileNotFoundError):
  
    f = open(btrack_init_path.replace('h5','pkl'),'rb')
    print('Opening original tracking.')

data, properties, graph = pickle.load(f)

Opening original tracking.


## Visualize

In [61]:
viewer = napari.Viewer()

# layer 0
viewer.add_image(myIm,colormap='gray',contrast_limits=(0, 2000),opacity = 1)

# layer 1
track_layer=viewer.add_tracks(data, properties=properties, graph=graph,name='tracking')
track_layer.display_id=True
#viewer.add_tracks(data_acc, properties=properties_acc, graph=graph_acc)

viewer.add_points(swapPoints,name='modPoints',face_color='green',ndim=3)

# layer 4
layer_inProgress = viewer.add_labels(inProgressTracks,name='inProgressStack',opacity = 0.5)
# layer 5
layer_accepted = viewer.add_labels(acceptedTracks,name='acceptedStack',opacity = 0.5)

In [132]:
# swapping tracks based on 4 points

@viewer.bind_key('r',overwrite=True)
def swap_points(viewer):

    swapPoints = viewer.layers['swapPoints'].data.astype(int)

    # check that you get a good set of swapPoints
    lenTest = (len(swapPoints) == 4)

    if lenTest:

        framesToSwap = list(set(swapPoints[:,0]))
        framesToSwap.sort()
        frameTest = (framesToSwap[1] == framesToSwap[0]+1)


        # read in in-progress tracks
        inProgressTracks = viewer.layers['inProgressStack'].data

        # check tracks
        myTracks = []
        for myPoint in swapPoints:

            myTracks.append(inProgressTracks[tuple(myPoint)])

        trackTest = (len(set(myTracks)) == 2)

        if (lenTest and frameTest and trackTest):

            # get tracks numbers
            track_1 = list((set(myTracks)))[0] 
            track_2 = list((set(myTracks)))[1]

            frame_change = np.max(framesToSwap)


            inProgressTracks[frame_change:,:,:][inProgressTracks[frame_change:,:,:]==track_1]=4095
            inProgressTracks[frame_change:,:,:][inProgressTracks[frame_change:,:,:]==track_2]=track_1
            inProgressTracks[frame_change:,:,:][inProgressTracks[frame_change:,:,:]==4095]=track_2

            # change labels layer
            viewer.layers.pop('inProgressStack')
            viewer.add_labels(inProgressTracks,name='inProgressStack',opacity = 0.5)


            # change graph
            children_1 = set(data[((properties['parent']==track_1) & (properties['generation']>0)),0])

            for child in children_1:

                graph[int(child)] = track_2

            children_2 = set(data[((properties['parent']==track_2) & (properties['generation']>0)),0])

            for child in children_2:

                graph[int(child)] = track_1


            # change tracking data
            track1_change = ((data[:,0]==track_1) & (data[:,1]>=frame_change))
            track2_change = ((data[:,0]==track_2) & (data[:,1]>=frame_change))

            data[track2_change,0] = track_1
            data[track1_change,0] = track_2

            # change info about the parent 
            track1_change = ((properties['parent']==track_1) & (data[:,1]>=frame_change))
            track2_change = ((properties['parent']==track_2) & (data[:,1]>=frame_change))

            properties['parent'][track1_change] = track_2
            properties['parent'][track2_change] = track_1

            # change info about the root
            root1 = properties['root'][track1_change][0]
            root2 = properties['root'][track2_change][0]

            track1_change = ((properties['root']==root1) & (data[:,1]>=frame_change))
            track2_change = ((properties['root']==root2) & (data[:,1]>=frame_change))

            properties['root'][track1_change] = root2
            properties['root'][track2_change] = root1

            # change info about the generation
            gen1 = properties['generation'][(properties['t']==(frame_change-1)) & (data[:,0]==track_1)]
            gen2 = properties['generation'][(properties['t']==(frame_change-1)) & (data[:,0]==track_2)]

            properties['generation'][track1_change] = properties['generation'][track1_change] + gen2 - gen1
            properties['generation'][track2_change] = properties['generation'][track2_change] + gen1 - gen2



            # update tracking data
            viewer.layers['data'].data = data
            viewer.layers['data'].properties = properties
            viewer.layers['data'].graph = graph

            # clean swap points
            viewer.layers['swapPoints'].data = []

            viewer.status='Tracks have been swapped.'

        else:

            viewer.status='Swap points are incorrect.'
    else:

        viewer.status='Swap points are incorrect.'

C:\ProgramData\Anaconda3\envs\cellpose\lib\site-packages\napari\layers\tracks\tracks.py:391: UserWarning: Previous color_by key 't' not present in new properties. Falling back to track_id
  UserWarning,


In [64]:
# right click for moving from inProgress to accepted layer
# around 5s 

@layer_inProgress.mouse_drag_callbacks.append
def prog_2_accept(layer, event):

    if(event.button == 2):
        
        # look up cursor position
        x = int(viewer.cursor.position[1])
        y = int(viewer.cursor.position[2])
        
        # get data
        inProgressTracks = layer_inProgress.data
        acceptedTracks = layer_accepted.data
        
        # check which cell was clicked
        myTrackNum = inProgressTracks[viewer.dims.current_step[0],x,y]
        
        if myTrackNum > 0:
            
            # move the clicked cell to accepted
            tempSel = (inProgressTracks==myTrackNum)
            acceptedTracks[tempSel]=myTrackNum
            inProgressTracks[tempSel]=0
            
            # update viewer    
            layer_inProgress.data = inProgressTracks
            layer_accepted.data = acceptedTracks

In [63]:
# transfer modification of labels into tracks layer

# at the moment one change at a time + key
# -> in a separate thread 
# -> take this frame, run regionprops
# -> check changes
# -> replace regionprops in the table

# lenght of history probably is not sufficient as it needs to look for a specific pattern of changes

regProps = ['label', 'area','centroid','major_axis_length','minor_axis_length']

changeIndex=[]

@viewer.bind_key('u',overwrite=True)
def update_inProgress(viewer):
    
    # add checking that inProgress layer is the active one
    
    data = viewer.layers['tracking'].data
    properties = viewer.layers['tracking'].properties
    
    # get the position
    myT = viewer.dims.current_step[0]
    
    # define objects to change 
    changeIndex = (data[:,1]==myT)

    # find new features of all the objects in the frame
    cellData = pd.DataFrame(measure.regionprops_table(inProgressTracks[myT,:,:], properties=regProps))
    cellData['t'] = myT
    cellData['state'] = 5
    cellData['track_id'] = cellData['label']
    
    # find info for new labels if existing
    labels_info=[np.where(data[:,0]==x)[0][0] if len(np.where(data[:,0]==x)[0])>0 else np.NaN for x in cellData.label]

    for f in ['parent','generation','root']:

        cellData[f] = [properties[f][x] if x==x else np.NaN for x in labels_info]

    # modify data of the track layer
    frameData = np.array(cellData.loc[:,['label','t','centroid-0','centroid-1']])

    data = np.delete(data,changeIndex,axis=0)

    data = np.vstack([data, frameData])
    
    # modify properties of the track layer
    for tProp in properties.keys():

        properties[tProp] = np.delete(properties[tProp],changeIndex)
        properties[tProp] = np.append(properties[tProp], cellData[tProp])

    # change tracks layer
    viewer.layers['tracking'].data = data
    viewer.layers['tracking'].properties = properties

    # change status
    viewer.status = f'Frame {myT} was modified.' 
    
# -> for changed traces re-generate small movie

C:\ProgramData\Anaconda3\envs\cellpose\lib\site-packages\napari\layers\tracks\tracks.py:391: UserWarning: Previous color_by key 't' not present in new properties. Falling back to track_id
  UserWarning,
C:\ProgramData\Anaconda3\envs\cellpose\lib\site-packages\napari\layers\tracks\tracks.py:391: UserWarning: Previous color_by key 't' not present in new properties. Falling back to track_id
  UserWarning,


In [ ]:
# add a button to Napari interface

In [68]:
@track_layer.bind_key('s',overwrite=True)
def update_inProgress(viewer):

    viewer_small = napari.Viewer()
    myTrack = 1 
    
    # create gallery
    
    viewer_small.add_image(myGallery,colormap='gray',contrast_limits=(0, 2000),opacity = 1)

In [149]:
len(np.where(data[:,0]==myTrack)[0])

361

In [208]:
#%%time

# create gallery
# there is an option to cut with or without surronding (at the moment it's with)

myTrack = 12
imSize = 100

data = track_layer.data

# calculate how many images there will be
imNum = np.sum(data[:,0]==myTrack)

# cut images to a small stack
# here deal with edge effects
smallIm = np.zeros([imNum,imSize,imSize]).astype('uint16')
smallLabel = smallIm.copy()

startFrame = int(np.min(data[data[:,0]==myTrack,1]))

for myInd in np.where(data[:,0]==myTrack)[0]:
    
    x = int(data[myInd,2])
    y = int(data[myInd,3])
    myFrame = int(data[myInd,1])
    
    t = myFrame - startFrame
    
    smallIm[t,:,:] = myIm[myFrame,x-int(imSize/2):x+int(imSize/2),y-int(imSize/2):y+int(imSize/2)]
    smallLabel[t,:,:] = inProgressTracks[myFrame,x-int(imSize/2):x+int(imSize/2),y-int(imSize/2):y+int(imSize/2)]
    
# make gallery out of stack

if np.sqrt(imNum).is_integer():
    
    col = row = int(np.sqrt(imNum))

else:
    
    col = int(np.sqrt(imNum))
    row = int(np.floor(imNum/col)+1)
            
# create canvas
myGallery = np.zeros([row*imSize,col*imSize]).astype('uint16')
myGalleryLabel = myGallery.copy()

# put images into canvas
for pair in product(range(row),range(col)):
        
        i = pair[1] # column iterator (fast)
        j = pair[0] # row iterator (slow)
        
        myFrame = j*col+i
        
        if myFrame<imNum:
        
            myGallery[j*imSize:(j+1)*imSize,i*imSize:(i+1)*imSize] = smallIm[myFrame,:,:] 
            myGalleryLabel[j*imSize:(j+1)*imSize,i*imSize:(i+1)*imSize] = smallLabel[myFrame,:,:]
        else:
            break

In [145]:
try:
    viewer_stack.close()
except:
    pass

In [209]:
try:
    viewer_stack.close()
except:
    pass

viewer_stack = napari.Viewer()

viewer_stack.add_image(smallIm,colormap='gray',contrast_limits=(0, 2000),opacity = 1,name=f'track {myTrack}')
viewer_stack.add_labels(smallLabel,name='inProgressStack',opacity = 0.5)

<Labels layer 'inProgressStack' at 0x2338c820048>

In [210]:
try:
    viewer_gal.close()
except:
    pass

viewer_gal = napari.Viewer()

viewer_gal.add_image(myGallery,colormap='viridis',contrast_limits=(0, 2000),opacity = 1)
viewer_gal.add_labels(myGalleryLabel,name='inProgressStack',opacity = 0.5)

<Labels layer 'inProgressStack' at 0x2338c1fb0c8>

In [ ]:
# create a button (or a whole menu) for these gallery views
# add numbering of the frames to gallery
# add graphs to other viewers

In [ ]:
# what to do with ghost objects

In [199]:
data[data[:,0]==1092,:]

array([[1092.        ,  318.        , 2162.92535971, 2026.29406475],
       [1092.        ,  319.        , 2162.92535971, 2026.29406475],
       [1092.        ,  320.        , 2139.88372093, 2058.0455814 ]])

In [201]:
data[(data[:,0]==7) & (data[:,1]==318),:]

array([[   7.        ,  318.        , 2172.5382482 , 2015.13455069]])

In [ ]:
# for faster checking of traces you need a galery
# the other option would be also to look at a trace, best DHB

# small movies should be generated only for traces longer than n frames

In [ ]:
# key binding for linking function

# check if parent have different offspring

# no other offspring
# connect points

# other offspring
# connect
# add to the graph

# what if objects didn;t

In [11]:
%%time
properties = ['label', 'area','centroid','orientation','major_axis_length','minor_axis_length','bbox','image']
cellData = pd.DataFrame(measure.regionprops_table(inProgressTracks[100,:,:], properties=properties))

Wall time: 102 ms


In [17]:
def print_layer_name(layer,event):
    print('hello')
    print(f"{event.source.name} changed its data!")

In [12]:
# from hooking up your own events
# not working at the moment

def print_layer_name(event):
    print(f"{event.source.name} changed its data!")

layer_inProgress.events.refresh.connect(print_layer_name)

<function __main__.print_layer_name(event)>

In [15]:
layer_inProgress.events

In [30]:
%%time
myTrackNum = 89
tempSel = (inProgressTracks==myTrackNum)
acceptedTracks[tempSel]=myTrackNum
inProgressTracks[tempSel]=0

Wall time: 5.37 s


In [31]:
tempSel.shape

(361, 2765, 2765)

In [22]:
%%time
myTrackNum = 65
t=[myTrackNum in x for x in inProgressTracks]

Wall time: 2.46 s


In [17]:
%time
test=layer_inProgress.data

Wall time: 0 ns


In [11]:
test = viewer.layers['inProgressStack'].data

In [18]:
test.shape

(361, 2765, 2765)

In [ ]:
test == inProgressTracks

In [21]:
t=viewer.layers['inProgressStack']._load_history()

TypeError: _load_history() missing 2 required positional arguments: 'before' and 'after'

In [20]:
t

<bound method Labels._load_history of <Labels layer 'inProgressStack' at 0x1b669ced408>>

In [17]:
viewer.layers['inProgressStack'].events.data()

<Event blocked=False handled=False native=None source=None sources=[] type=data>

In [18]:
# from hooking up your own events
# not working at the moment

def print_layer_name(event):
    print(f"{event.source.name} changed its data!")

viewer.layers['inProgressStack'].events.set_data.connect(print_layer_name)


<function __main__.print_layer_name(event)>

inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStac

inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStac

inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStack changed its data!
inProgressStac

In [56]:
def _on_removed(event):
    layer = event.value
    print("removed: " + layer.name)

viewer.layers.events.removed.connect(_on_removed)

<function __main__._on_removed(event)>

removed: mitosis
removed: acceptedStack


In [58]:
event

NameError: name 'event' is not defined

## Save annotated points

In [62]:
# save annotations
donePoints = viewer.layers['correct'].data.astype(int)
np.save(os.path.join(pointDir,donePointsPath),donePoints)

mitosisPoints = viewer.layers['mitosis'].data.astype(int)
np.save(os.path.join(pointDir,mitosisPointsPath),mitosisPoints)

## Resolve swap points

In [14]:
@viewer.bind_key('r',overwrite=True)
def swap_points(viewer):
    
    swapPoints = viewer.layers['swapPoints'].data.astype(int)

    # check that you get a good set of swapPoints
    lenTest = (len(swapPoints) == 4)

    if lenTest:

        framesToSwap = list(set(swapPoints[:,0]))
        framesToSwap.sort()
        frameTest = (framesToSwap[1] == framesToSwap[0]+1)

        # read in in-progress tracks
        inProgressTracks = viewer.layers['inProgressStack'].data

        # check tracks
        myTracks = []
        for myPoint in swapPoints:

            myTracks.append(inProgressTracks[tuple(myPoint)])

        trackTest = (len(set(myTracks)) == 2)

        if (lenTest and frameTest and trackTest):

            # get tracks numbers
            track_1 = list((set(myTracks)))[0] 
            track_2 = list((set(myTracks)))[1]

            frame_change = np.max(framesToSwap)

            inProgressTracks[frame_change:,:,:][inProgressTracks[frame_change:,:,:]==track_1]=4095
            inProgressTracks[frame_change:,:,:][inProgressTracks[frame_change:,:,:]==track_2]=track_1
            inProgressTracks[frame_change:,:,:][inProgressTracks[frame_change:,:,:]==4095]=track_2

            # change labels layer
            viewer.layers.pop('inProgressStack')
            viewer.add_labels(inProgressTracks,name='inProgressStack',opacity = 0.5)

            '''
            ###########################################################################
            # this will be a good option if you could create a fast way to encode this data from layer to structure
            
            # change tracking data
            data,properties,graph = getTrackingFromLayer(inProgressTracks)
            
            # update tracking data
            viewer.layers['data'].data = data
            viewer.layers['data'].properties = properties
            
            # otherwise just modify the change
            # but the change propagates through generations!!!
            # so you may need to change parent for kids 
            
            ############################################################################
            '''
            
            # clean swap points
            viewer.layers['swapPoints'].data = []

            viewer.status='Tracks have been swapped.'

        else:

            viewer.status='Swap points are incorrect.'
    else:

        viewer.status='Swap points are incorrect.'

In [131]:
swapPoints = viewer.layers['swapPoints'].data.astype(int)
swapPoints

array([[  25, 1138, 1073],
       [  25, 1121, 1163],
       [  26, 1142, 1066],
       [  26, 1120, 1161]])

In [129]:
#swapPoints = viewer.layers['swapPoints'].data.astype(int)
swapPoints

array([[  25, 1142, 1074],
       [  25, 1121, 1163],
       [  26, 1139, 1068],
       [  26, 1123, 1160]])

In [135]:
swapPoints = viewer.layers['swapPoints'].data.astype(int)


# check that you get a good set of swapPoints
lenTest = (len(swapPoints) == 4)

if lenTest:

    framesToSwap = list(set(swapPoints[:,0]))
    framesToSwap.sort()
    frameTest = (framesToSwap[1] == framesToSwap[0]+1)


    # read in in-progress tracks
    inProgressTracks = viewer.layers['inProgressStack'].data

    # check tracks
    myTracks = []
    for myPoint in swapPoints:

        myTracks.append(inProgressTracks[tuple(myPoint)])

    trackTest = (len(set(myTracks)) == 2)

    if (lenTest and frameTest and trackTest):

        # get tracks numbers
        track_1 = list((set(myTracks)))[0] 
        track_2 = list((set(myTracks)))[1]

        frame_change = np.max(framesToSwap)

              
children_1 = set(data[((properties['parent']==track_1) & (properties['generation']>0)),0])
children_2 = set(data[((properties['parent']==track_2) & (properties['generation']>0)),0])

In [136]:
print(f'frame change {frame_change}')

print(track_1)

for child in children_1:
    print(child)
    print(f'parent {graph[int(child)]}')

print(track_2)

for child in children_2:
    print(child)
    print(f'parent {graph[int(child)]}')

frame change 26
16
768.0
parent [16]
767.0
parent [16]
13
1128.0
parent [13]
1117.0
parent [13]


In [134]:
print(f'frame change {frame_change}')

print(track_1)

for child in children_1:
    print(child)
    print(f'parent {graph[int(child)]}')

print(track_2)

for child in children_2:
    print(child)
    print(f'parent {graph[int(child)]}')

frame change 26
16
1128.0
parent [13]
1117.0
parent [13]
13
768.0
parent [16]
767.0
parent [16]


In [111]:
children_1

{767.0, 768.0}

In [95]:
for child in children_1:
    
    print(graph[int(child)])

[13]
[13]


In [93]:
graph[1128]

[13]

In [80]:
properties['generation'][data[:,0]==767]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [76]:
np.sum(properties['parent']==1117)

0

In [43]:
# change tracking data
track1_change = ((data[:,0]==track_1) & (data[:,1]>=frame_change))
track2_change = ((data[:,0]==track_2) & (data[:,1]>=frame_change))

In [52]:
graph

{188: [175],
 189: [175],
 199: [19],
 203: [19],
 216: [186],
 217: [186],
 241: [223],
 242: [223],
 254: [227],
 262: [227],
 263: [146],
 267: [146],
 270: [141],
 271: [141],
 294: [272],
 295: [272],
 331: [100],
 332: [100],
 383: [37],
 384: [37],
 390: [354],
 392: [354],
 398: [374],
 399: [374],
 510: [245],
 511: [245],
 514: [161],
 518: [512],
 526: [161],
 531: [512],
 532: [106],
 533: [106],
 540: [22],
 542: [21],
 544: [21],
 550: [7],
 551: [7],
 552: [22],
 559: [168],
 566: [318],
 567: [318],
 570: [168],
 574: [417],
 587: [417],
 617: [304],
 618: [533],
 619: [533],
 629: [554],
 632: [304],
 640: [554],
 663: [89],
 664: [89],
 675: [105],
 680: [105],
 697: [12],
 700: [12],
 710: [143],
 711: [143],
 743: [728],
 744: [728],
 759: [126],
 760: [126],
 767: [16],
 768: [16],
 770: [159],
 772: [159],
 798: [782],
 799: [782],
 803: [142],
 804: [142],
 846: [41],
 847: [41],
 853: [85],
 854: [85],
 894: [737],
 895: [737],
 940: [912],
 941: [912],
 960: [1

In [45]:
np.sum(track1_change)

39

In [39]:
swapPoints = viewer.layers['swapPoints'].data.astype(int)


# check that you get a good set of swapPoints
lenTest = (len(swapPoints) == 4)

if lenTest:

    framesToSwap = list(set(swapPoints[:,0]))
    framesToSwap.sort()
    frameTest = (framesToSwap[1] == framesToSwap[0]+1)


    # read in in-progress tracks
    inProgressTracks = viewer.layers['inProgressStack'].data

    # check tracks
    myTracks = []
    for myPoint in swapPoints:

        myTracks.append(inProgressTracks[tuple(myPoint)])

    trackTest = (len(set(myTracks)) == 2)

    if (lenTest and frameTest and trackTest):

        # get tracks numbers
        track_1 = list((set(myTracks)))[0] 
        track_2 = list((set(myTracks)))[1]

        frame_change = np.max(framesToSwap)

In [41]:
frame_change

26

In [ ]:
properties[changeFeature]

In [46]:
featuresList =['parent','generation','root']

for changeFeature in featuresList:

    temp1 = properties[changeFeature][track1_change][0]
    temp2 = properties[changeFeature][track2_change][0]
    print(temp1)
    print(temp2)

16
13
0
0
16
13


In [47]:
properties['parent']

array([   1,    1,    1, ..., 3978, 3979, 3980])

In [49]:
np.max(properties['generation'])

2

In [ ]:
track1_change = ((data[:,0]==track_1) and (data[:,1]>=frame_change))
track2_change = ((data[:,0]==track_2) and (data[:,1]>=frame_change))

data[track2_change,:] = track_1
data[track1_change,:] = track_2

featuresList =['parent','generation','root']

for changeFeature in featuresList:

    temp1 = properties[changeFeature][track1_change] 
    temp2 = properties[changeFeature][track2_change]

    properties[changeFeature][track1_change] = temp2
    properties[changeFeature][track2_change] = temp1

array([   1,    1,    1, ..., 3978, 3979, 3980])

In [18]:
properties

{'t': array([  0,   1,   2, ..., 425, 425, 425]),
 'state': array([5, 5, 5, ..., 5, 5, 5]),
 'generation': array([0, 0, 0, ..., 0, 0, 0]),
 'root': array([   1,    1,    1, ..., 3978, 3979, 3980]),
 'parent': array([   1,    1,    1, ..., 3978, 3979, 3980]),
 'area': array([594., 554., 539., ..., 104., 632., 130.]),
 'major_axis_length': array([30.93546676, 30.50784398, 29.9256982 , ..., 16.59471649,
        38.55793435, 15.47378568]),
 'minor_axis_length': array([24.64765807, 23.34283118, 23.26908727, ...,  8.53638228,
        21.1229483 , 10.9748531 ])}

In [85]:
framesToSwap = list(set(swapPoints[:,0]))
framesToSwap.sort()
frameTest = (framesToSwap[1] == framesToSwap[0]+1)
frameTest

True

In [74]:
swapPoints = viewer.layers['swapPoints'].data.astype(int)

In [81]:
swapPoints

array([[ 292,  745, 1418],
       [ 292,  767, 1423],
       [ 293,  743, 1425],
       [ 293,  772, 1428]])

In [80]:
len(set(swapPoints[:,0])) == 2

2

In [69]:
viewer.add_labels(inProgressTracks,name='inProgressStack',opacity = 0.5)

<Labels layer 'inProgressStack' at 0x2842bf50f48>

In [65]:
viewer.layers

[<Image layer 'myIm' at 0x28427515ac8>, <Tracks layer 'data' at 0x28427204908>, <Points layer 'correct' at 0x2842720ae88>, <Points layer 'mitosis' at 0x28427223948>, <Points layer 'swapPoints' at 0x2842cb34688>, <Labels layer 'swapped' at 0x28428749b48>, <Labels layer 'inProgressStack' at 0x2830f439a48>, <Labels layer 'inProgressStack [1]' at 0x2842ad7d808>]

In [68]:
viewer.layers.pop('inProgressStack [1]')

<Labels layer 'inProgressStack [1]' at 0x2842ad7d808>

In [53]:
inProgressTracks = viewer.layers['inProgressStack'].data

In [54]:
swapPoints = viewer.layers['swapPoints'].data.astype(int)
swapPoints = pd.DataFrame(swapPoints,columns=['t','x','y'])
swapPoints

,t,x,y
0,292,745,1418
1,292,767,1423
2,293,743,1425
3,293,772,1428


In [55]:
myTracks = []
for ind,myPoint in swapPoints.iterrows():

    myTracks.append(inProgressTracks[tuple(myPoint)])
    swapPoints.loc[ind,'track'] = myTrackNum

In [56]:
swapPoints

,t,x,y,track
0,292,745,1418,132.0
1,292,767,1423,1365.0
2,293,743,1425,1365.0
3,293,772,1428,132.0


In [57]:
frame_change = 293
track_1 = 132
track_2 = 1365

In [58]:
inProgressTracks[frame_change:,:,:][inProgressTracks[frame_change:,:,:]==track_1]=4095
inProgressTracks[frame_change:,:,:][inProgressTracks[frame_change:,:,:]==track_2]=track_1
inProgressTracks[frame_change:,:,:][inProgressTracks[frame_change:,:,:]==4095]=track_2

In [59]:
viewer.add_labels(inProgressTracks,opacity = 0.5,name='swapped')

<Labels layer 'swapped' at 0x28428749b48>

In [ ]:
viewer.layers.pop(4)

## Save label data

In [60]:
inProgressTracks = viewer.layers['swapped'].data
acceptedTracks = viewer.layers['acceptedStack'].data

myLabels = inProgressTracks + acceptedTracks
myLabels.shape

(426, 1946, 1946)

In [61]:
%%time

for j in range(startFrame,stopFrame):
        
    imsave(os.path.join(myDirTracks,myFile.replace('.tif',f'_{str(j).zfill(3)}_track.tif')),myLabels[j].astype('uint16'))

Wall time: 17.6 s


## Calculate regionprops for modified frames

## Recalculate tracking

In [54]:
objects = btrack.utils.segmentation_to_objects(inProgressTracks, properties=('area', 'major_axis_length','minor_axis_length'))

[INFO][2021/05/24 05:04:20 PM] Localizing objects from segmentation...
[INFO][2021/05/24 05:05:49 PM] Objects are of type: <class 'dict'>
[INFO][2021/05/24 05:05:52 PM] ...Found 191312 objects in 426 frames.


In [55]:
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:

  # configure the tracker using a config file
  tracker.configure_from_file(modelPath)

  # append the objects to be tracked
  tracker.append(objects)

  # set the volume (Z axis volume is set very large for 2D data)
  tracker.volume=((0, inProgressTracks.shape[1]), (0, inProgressTracks.shape[2]), (-1e5, 1e5))

  # track them (in interactive mode)
  tracker.track_interactive(step_size=100)

  # generate hypotheses and run the global optimizer
  tracker.optimize()

  # store the data in an HDF5 file
  tracker.export(btrack_mod_path, obj_type='obj_type_1')

  # get the tracks as a python list
  tracks = tracker.tracks

  # optional: get the data in a format for napari
  data, properties, graph = tracker.to_napari(ndim=2)

[INFO][2021/05/24 05:05:52 PM] Loaded btrack: C:\ProgramData\Anaconda3\envs\cellpose\lib\site-packages\btrack\libs\libtracker.DLL
[INFO][2021/05/24 05:05:52 PM] btrack (v0.4.1) library imported
[INFO][2021/05/24 05:05:52 PM] Setting max XYZ search radius to: 100
[INFO][2021/05/24 05:05:52 PM] Starting BayesianTracker session
[INFO][2021/05/24 05:05:52 PM] Loading configuration file: D:\BARC\Sonja_2021\b_track\cell_config.json
[INFO][2021/05/24 05:05:52 PM] Loading motion model: b'cell_motion'
[INFO][2021/05/24 05:05:52 PM] Objects are of type: <class 'list'>
[INFO][2021/05/24 05:05:52 PM] Set volume to ((0, 1946), (0, 1946), (-100000.0, 100000.0))
[INFO][2021/05/24 05:05:52 PM] Starting tracking... 
[INFO][2021/05/24 05:05:52 PM] Tracking objects in frames 0 to 99 (of 426)...
[INFO][2021/05/24 05:05:58 PM]  - Timing (Bayesian updates: 82.00ms, Linking: 3.00ms)
[INFO][2021/05/24 05:05:58 PM]  - Probabilities (Link: 1.00000, Lost: 1.00000)
[INFO][2021/05/24 05:05:58 PM]  - Stats (Active:

In [51]:
# build tracking from accepted data
# how to build generation, root, parent data

In [56]:
np.max(properties['generation'])

2

In [48]:


objects = btrack.utils.segmentation_to_objects(acceptedTracks, properties=('area', 'major_axis_length','minor_axis_length'))

[INFO][2021/05/24 04:57:34 PM] Localizing objects from segmentation...
[INFO][2021/05/24 04:57:46 PM] Objects are of type: <class 'dict'>
[INFO][2021/05/24 04:57:46 PM] ...Found 852 objects in 426 frames.


In [49]:
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:

  # configure the tracker using a config file
  tracker.configure_from_file(modelPath)

  # append the objects to be tracked
  tracker.append(objects)

  # set the volume (Z axis volume is set very large for 2D data)
  tracker.volume=((0, acceptedTracks.shape[1]), (0, acceptedTracks.shape[2]), (-1e5, 1e5))

  # track them (in interactive mode)
  tracker.track_interactive(step_size=100)

  # generate hypotheses and run the global optimizer
  tracker.optimize()

  # optional: get the data in a format for napari
  data_acc, properties_acc, graph_acc = tracker.to_napari(ndim=2)

[INFO][2021/05/24 04:58:38 PM] Loaded btrack: C:\ProgramData\Anaconda3\envs\cellpose\lib\site-packages\btrack\libs\libtracker.DLL
[INFO][2021/05/24 04:58:38 PM] btrack (v0.4.1) library imported
[INFO][2021/05/24 04:58:38 PM] Setting max XYZ search radius to: 100
[INFO][2021/05/24 04:58:38 PM] Starting BayesianTracker session
[INFO][2021/05/24 04:58:38 PM] Loading configuration file: D:\BARC\Sonja_2021\b_track\cell_config.json
[INFO][2021/05/24 04:58:38 PM] Loading motion model: b'cell_motion'
[INFO][2021/05/24 04:58:38 PM] Objects are of type: <class 'list'>
[INFO][2021/05/24 04:58:38 PM] Set volume to ((0, 1946), (0, 1946), (-100000.0, 100000.0))
[INFO][2021/05/24 04:58:38 PM] Starting tracking... 
[INFO][2021/05/24 04:58:38 PM] Tracking objects in frames 0 to 99 (of 426)...
[INFO][2021/05/24 04:58:38 PM]  - Timing (Bayesian updates: 0.00ms, Linking: 0.00ms)
[INFO][2021/05/24 04:58:38 PM]  - Probabilities (Link: 1.00000, Lost: 0.94720)
[INFO][2021/05/24 04:58:38 PM]  - Stats (Active: 

In [19]:
with open(btrack_mod_path.replace('h5','pkl'),'wb') as f:
    pickle.dump([data,properties,graph],f)

In [45]:
np.sum(properties['t']==data[:,1])

193023

In [47]:
properties

{'t': array([  0,   1,   2, ..., 425, 425, 425]),
 'state': array([5, 5, 5, ..., 5, 5, 5]),
 'generation': array([0, 0, 0, ..., 0, 0, 0]),
 'root': array([   1,    1,    1, ..., 3960, 3961, 3962]),
 'parent': array([   1,    1,    1, ..., 3960, 3961, 3962]),
 'major_axis_length': array([35.47797919, 35.07999452, 37.50516787, ..., 16.59471649,
        38.55793435, 15.47378568]),
 'area': array([748., 736., 805., ..., 104., 632., 130.]),
 'minor_axis_length': array([27.19400299, 27.02497758, 27.7785213 , ...,  8.53638228,
        21.1229483 , 10.9748531 ])}

In [46]:
properties['t'].shape

(193023,)

In [35]:
donePoints = viewer.layers['correct'].data.astype(int)
donePoints

array([[   0,  918, 1096]])

In [29]:
donePoints = viewer.layers['correct'].data.astype(int)
donePoints

for myPoint in donePoints:
    
    # get the track that was marked as done
    myTrackNum = inProgressTracks[tuple(myPoint)]
    
    acceptedTracks[inProgressTracks==myTrackNum]=myTrackNum
    inProgressTracks[inProgressTracks==myTrackNum]=0

83


In [ ]:
acceptedTracks[inProgressTracks==myTrackNum]=myTrackNum
inProgressTracks[inProgressTracks==myTrackNum]=0

In [27]:
# get modified labels
inProgressTracks = viewer.layers['inProgressStack'].data.astype('uint16')
inProgressTracks.shape

(426, 1946, 1946)

In [28]:
# get modified labels
acceptedTracks = viewer.layers['acceptedStack'].data.astype('uint16')
acceptedTracks.shape

KeyError: "'acceptedStack' is not in list"

Here you can maybe do it smarter and recalculate only if there was a change.

In [73]:
%%time

# settings for properties to calculate
regionProperties = ['label', 'area','centroid','major_axis_length','minor_axis_length','bbox','image']

cellDataList=[]

i=startFrame
for labels_2D in inProgressTracks:

    # calculate properties of regions
    cellData = pd.DataFrame(measure.regionprops_table(labels_2D, properties=regionProperties))

    # add info of these measurements
    cellData['t'] = i

    cellDataList.append(cellData)
    i = i + 1

# put all together
cellDataAll = pd.concat(cellDataList,ignore_index=True)

Wall time: 1min 31s


In [74]:
cellDataAll['x']=cellDataAll['centroid-1'].astype(int)
cellDataAll['y']=cellDataAll['centroid-0'].astype(int)
cellDataAll.head()

,label,area,centroid-0,centroid-1,major_axis_length,minor_axis_length,bbox-0,bbox-1,bbox-2,bbox-3,image,t,x,y
0,1,594,1136.489899,1389.333333,30.935467,24.647658,1125,1372,1149,1405,"[[False, False, False, False, False, False, Fa...",0,1389,1136
1,2,429,1035.256410,1021.067599,31.773448,18.266324,1026,1010,1050,1036,"[[False, False, False, False, False, False, Fa...",0,1021,1035
2,3,432,1046.462963,374.643519,31.138550,18.234134,1031,365,1062,386,"[[False, False, False, False, False, True, Tru...",0,374,1046
3,4,568,1048.593310,1347.799296,33.461723,22.048148,1036,1332,1062,1363,"[[False, False, False, False, False, False, Fa...",0,1347,1048
4,5,501,1051.788423,956.137725,27.223351,23.667042,1041,944,1064,970,"[[False, False, False, False, False, False, Fa...",0,956,1051


In [64]:
for myPoint in donePoints:
    
    # get the track that was marked as done
    myTrackNum = modTracks[tuple(myPoint)]
    
    #myTrack = tracks[[x.ID for x in tracks].index(myTrackNum)]
    
    # check if it has a mito mark and don't label as 'done' beyond this point
    
    cellDataAll.loc[cellDataAll.label == myTrackNum,'done']=True

In [ ]:
# find labels with mito mark


In [65]:
np.sum(cellDataAll.done==True)

426

In [66]:
allData = cellDataAll.copy()

## Reconstruct accepted and work in progress

In [67]:
%%time

acceptedStack=[]
for i in range(stopFrame):
    
    # choose data from this frame
    selData = allData.loc[((allData.t==i) & (allData.done == True)),:]
    
    # create an empty image
    labelImage = np.zeros([myLabels.shape[1],myLabels.shape[2]]).astype('uint16')
    
    # add objects
    for ind,myCell in selData.iterrows():
        
        min_row = myCell['bbox-0']
        max_row = myCell['bbox-2']
        min_col = myCell['bbox-1']
        max_col = myCell['bbox-3']
        
        labelImage[min_row:max_row,min_col:max_col]=labelImage[min_row:max_row,min_col:max_col]+(myCell.image*myCell.label)
        
    acceptedStack.append(labelImage)

acceptedStack = np.array(acceptedStack)
acceptedStack.shape

Wall time: 8.66 s


(426, 1946, 1946)

In [68]:
acceptedStack = acceptedStack + acceptedTracks

In [69]:
%%time

inProgressStack=[]
for i in range(stopFrame):
    
    # choose data from this frame
    selData = allData.loc[((allData.t==i) & (allData.done != True)),:]
    
    # create an empty image
    labelImage = np.zeros([myLabels.shape[1],myLabels.shape[2]]).astype('uint16')
    
    # add objects
    for ind,myCell in selData.iterrows():
        
        min_row = myCell['bbox-0']
        max_row = myCell['bbox-2']
        min_col = myCell['bbox-1']
        max_col = myCell['bbox-3']
        
        labelImage[min_row:max_row,min_col:max_col]=labelImage[min_row:max_row,min_col:max_col]+(myCell.image*myCell.label)
        
    inProgressStack.append(labelImage)

inProgressStack = np.array(inProgressStack)
inProgressStack.shape

Wall time: 43 s


(426, 1946, 1946)

In [70]:
viewer = napari.Viewer()
viewer.add_image(myIm)
viewer.add_labels(acceptedStack)
viewer.add_labels(inProgressStack)
viewer.add_tracks(data, properties=properties, graph=graph)

viewer.add_points(donePoints,name='correct',face_color='black')

<Points layer 'correct' at 0x1cf5db59888>

In [57]:
viewer.add_points(donePoints,name='correct',face_color='black')

<Points layer 'correct' at 0x1ce4b1f19c8>

In [ ]:
tracksList=[]

for i in range(len(tracks)):
    
    # process track data
    t = tracks[i]

    track_temp = pd.DataFrame(data=np.array([t.t,t.x,t.y,t.dummy,t.properties['major_axis_length'],
                                             t.properties['minor_axis_length'],t.properties['area']]).T,
                           columns=('t','x','y','dummy','major_axis_length','minor_axis_length','area'))

    track_temp['ID'] = t.ID
    track_temp['generation'] = t.generation
    track_temp['parent'] = t.parent
    track_temp['root'] = t.root

    track_temp['x']=track_temp['x'].astype(int)
    track_temp['y']=track_temp['y'].astype(int)
    
    tracksList.append(track_temp)
    
# put all together
trackDataAll = pd.concat(tracksList,ignore_index=True)

In [87]:
mitosisPoints = viewer.layers['mitosis'].data.astype(int)
mitosisPoints

array([[  0, 776, 659]])

In [88]:
mitosisPoints

array([[  0, 776, 659]])

In [ ]:
modTracks[tuple(donePoints[0])]

In [38]:
%%time

#

imsave(os.path.join(dirMod,myFile.replace('.tif','_mod.tif')),modTracks)

Wall time: 17.7 s


In [52]:
donePoints = viewer.layers['done'].data.astype(int)

array([425, 600, 737])

In [58]:
tuple(donePoints[0])

(425, 600, 737)

In [53]:
doneTracks = modTracks[donePoints[0]]

IndexError: index 600 is out of bounds for axis 0 with size 426

In [59]:
modTracks[tuple(donePoints[0])]

116

In [17]:
graph

{190: [176],
 191: [176],
 199: [128],
 202: [128],
 213: [184],
 214: [184],
 242: [222],
 243: [222],
 258: [225],
 262: [225],
 263: [81],
 269: [81],
 270: [76],
 271: [76],
 292: [272],
 293: [272],
 332: [35],
 333: [35],
 383: [102],
 384: [102],
 387: [353],
 389: [353],
 398: [376],
 399: [376],
 408: [59],
 409: [59],
 510: [245],
 511: [245],
 517: [512],
 530: [512],
 531: [41],
 532: [41],
 542: [130],
 543: [130],
 551: [116],
 552: [116],
 557: [60],
 564: [317],
 565: [317],
 568: [60],
 576: [421],
 590: [421],
 616: [305],
 617: [532],
 618: [532],
 627: [553],
 629: [305],
 641: [553],
 661: [24],
 662: [24],
 675: [40],
 677: [40],
 699: [110],
 700: [110],
 708: [67],
 709: [67],
 748: [726],
 749: [726],
 756: [17],
 757: [17],
 767: [51],
 768: [51],
 769: [125],
 770: [125],
 798: [77],
 799: [77],
 801: [782],
 802: [782],
 844: [106],
 845: [106],
 852: [151],
 853: [151],
 896: [737],
 897: [737],
 933: [902],
 934: [902],
 961: [13],
 962: [271],
 963: [271]

In [28]:
[x.ID for x in tracks].index(1848)

1119

In [29]:
tracks[[x.ID for x in tracks].index(1848)]

,ID,t,x,y,z,parent,root,state,generation,dummy,area,minor_axis_length,major_axis_length
0,1848,198,818.802111,1258.738786,0.0,1848,1848,5,0,False,758,26.272827,37.201714


## Recalculate tracking

In [8]:
# get modified labels

myPath = os.path.join(dirMod,os.listdir(dirMod)[0])
print(myPath)

modTracks = imread(myPath)

D:\BARC\Sonja_2021\man_mod\20210414imagingmjxy03c1_bckCorr_mod.tif


In [39]:
savePath = r'D:\BARC\Martha_2021\Martha_tracks_v_01.h5'

In [40]:
objects = btrack.utils.segmentation_to_objects(modTracks, properties=('area', 'major_axis_length','minor_axis_length'))

[INFO][2021/05/20 01:19:43 PM] Localizing objects from segmentation...
[INFO][2021/05/20 01:21:13 PM] Objects are of type: <class 'dict'>
[INFO][2021/05/20 01:21:16 PM] ...Found 192165 objects in 426 frames.


In [41]:
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:

  # configure the tracker using a config file
  tracker.configure_from_file(modelPath)

  # append the objects to be tracked
  tracker.append(objects)

  # set the volume (Z axis volume is set very large for 2D data)
  tracker.volume=((0, modTracks.shape[1]), (0, modTracks.shape[2]), (-1e5, 1e5))

  # track them (in interactive mode)
  tracker.track_interactive(step_size=100)

  # generate hypotheses and run the global optimizer
  tracker.optimize()

  # store the data in an HDF5 file
  tracker.export(savePath, obj_type='obj_type_1')

  # get the tracks as a python list
  tracks = tracker.tracks

  # optional: get the data in a format for napari
  data, properties, graph = tracker.to_napari(ndim=2)

[INFO][2021/05/20 01:21:16 PM] Loaded btrack: C:\ProgramData\Anaconda3\envs\cellpose\lib\site-packages\btrack\libs\libtracker.DLL
[INFO][2021/05/20 01:21:16 PM] btrack (v0.4.1) library imported
[INFO][2021/05/20 01:21:16 PM] Setting max XYZ search radius to: 100
[INFO][2021/05/20 01:21:16 PM] Starting BayesianTracker session
[INFO][2021/05/20 01:21:16 PM] Loading configuration file: D:\BARC\Sonja_2021\b_track\cell_config.json
[INFO][2021/05/20 01:21:16 PM] Loading motion model: b'cell_motion'
[INFO][2021/05/20 01:21:16 PM] Objects are of type: <class 'list'>
[INFO][2021/05/20 01:21:16 PM] Set volume to ((0, 1946), (0, 1946), (-100000.0, 100000.0))
[INFO][2021/05/20 01:21:16 PM] Starting tracking... 
[INFO][2021/05/20 01:21:17 PM] Tracking objects in frames 0 to 99 (of 426)...
[INFO][2021/05/20 01:21:22 PM]  - Timing (Bayesian updates: 79.00ms, Linking: 2.00ms)
[INFO][2021/05/20 01:21:22 PM]  - Probabilities (Link: 1.00000, Lost: 1.00000)
[INFO][2021/05/20 01:21:22 PM]  - Stats (Active:

## Visualize

In [42]:
viewer = napari.Viewer()
viewer.add_image(myIm)
viewer.add_labels(modTracks)
viewer.add_tracks(data, properties=properties, graph=graph)

<Tracks layer 'data' at 0x2017b06abc8>